# Copyright Netherlands eScience Center <br>
** Function     : Download S2S data from ECMWF MARS** <br>
** Author       : Yang Liu ** <br>
** First Built  : 2020.07.17 ** <br>
** Last Update  : 2020.07.20 ** <br>
Description     : This notebook aims to download S2S data from ECMWF server. It includes both the control 
                  run and ensemble run from daily average branch.<br>
Return Values   : netCDF4 <br>
Caveat          : Before execute the code, the following steps should be complete:
1. Install ECMWF Key
The key could be retrived through the link, after login: https://api.ecmwf.int/v1/key/
For Windows, the key should be placed at the user folder (for instance, C:\Users\ESLT0068\.ecmwfapirc)
2. Install client library
The ecmwfapi python library could be accessed through:
'sudo pip install https://software.ecmwf.int/wiki/download/attachments/56664858/ecmwf-api-client-python.tgz'
"""

In [1]:
import os
import time as tttt
import numpy as np
from ecmwfapi import ECMWFDataServer
import logging

In [2]:
##############################   Logging setup  #######################################
# record the time for downloading the dataset
start_time = tttt.time()
#forecast_type = "ensemble" # control / ensemble
forecast_type = "control"
# Redirect all the console output to a file
#sys.stdout = open('/home/ESLT0068/WorkFlow/Core_Database_DeepLearn/ECMWFens/S2S{}/console.out'.format(forecast_type),'w+'))

# logging level 'DEBUG' 'INFO' 'WARNING' 'ERROR' 'CRITICAL'
logging.basicConfig(filename = '/home/ESLT0068/WorkFlow/Core_Database_DeepLearn/ECMWFens/S2S{}/history.log'.format(forecast_type),
                    filemode = 'w+', level = logging.DEBUG, format = '%(asctime)s - %(name)s - %(levelname)s - %(message)s')

In [3]:
# Input zone  #########################################
# specify output path for figures
output_path = '/home/ESLT0068/WorkFlow/Core_Database_DeepLearn/ECMWFens/S2S{}'.format(forecast_type)
# time
#target_time = [201501, 201502, 201503, 201504, 201505, 201506, 201507, 201508, 201509, 201510, 201511, 201512]
target_time = [201601, 201602, 201603, 201604, 201605, 201606, 201607, 201608, 201609, 201610, 201611, 201612]
#target_time = [201701, 201702, 201703, 201704, 201705, 201706, 201707, 201708, 201709, 201710, 201711, 201712]

date_string = {}
date_string["201501"] = "2015-01-01/2015-01-05/2015-01-08/2015-01-12/2015-01-15/2015-01-19/2015-01-22/2015-01-26/2015-01-29"
date_string["201502"] = "2015-02-02/2015-02-05/2015-02-09/2015-02-12/2015-02-16/2015-02-19/2015-02-23/2015-02-26"
date_string["201503"] = "2015-03-02/2015-03-05/2015-03-09/2015-03-12/2015-03-16/2015-03-19/2015-03-23/2015-03-26/2015-03-30"
date_string["201504"] = "2015-04-02/2015-04-06/2015-04-09/2015-04-13/2015-04-16/2015-04-20/2015-04-23/2015-04-27/2015-04-30"
date_string["201505"] = "2015-05-04/2015-05-07/2015-05-11/2015-05-14/2015-05-18/2015-05-21/2015-05-25/2015-05-28"
date_string["201506"] = "2015-06-01/2015-06-04/2015-06-08/2015-06-11/2015-06-15/2015-06-18/2015-06-22/2015-06-25/2015-06-29"
date_string["201507"] = "2015-07-02/2015-07-06/2015-07-09/2015-07-13/2015-07-16/2015-07-20/2015-07-23/2015-07-27/2015-07-30"
date_string["201508"] = "2015-08-03/2015-08-06/2015-08-10/2015-08-13/2015-08-17/2015-08-20/2015-08-24/2015-08-27/2015-08-31"
date_string["201509"] = "2015-09-03/2015-09-07/2015-09-10/2015-09-14/2015-09-17/2015-09-21/2015-09-24/2015-09-28"
date_string["201510"] = "2015-10-01/2015-10-05/2015-10-08/2015-10-12/2015-10-15/2015-10-19/2015-10-22/2015-10-26/2015-10-29"
date_string["201511"] = "2015-11-02/2015-11-05/2015-11-09/2015-11-12/2015-11-16/2015-11-19/2015-11-23/2015-11-26/2015-11-30"
date_string["201512"] = "2015-12-03/2015-12-07/2015-12-10/2015-12-14/2015-12-17/2015-12-21/2015-12-24/2015-12-28/2015-12-31"

date_string["201601"] = "2016-01-04/2016-01-07/2016-01-11/2016-01-14/2016-01-18/2016-01-21/2016-01-25/2016-01-28"
date_string["201602"] = "2016-02-01/2016-02-04/2016-02-08/2016-02-11/2016-02-15/2016-02-18/2016-02-22/2016-02-25/2016-02-29"
date_string["201603"] = "2016-03-03/2016-03-07/2016-03-10/2016-03-14/2016-03-17/2016-03-21/2016-03-24/2016-03-28/2016-03-31"
date_string["201604"] = "2016-04-04/2016-04-07/2016-04-11/2016-04-14/2016-04-18/2016-04-21/2016-04-25/2016-04-28"
date_string["201605"] = "2016-05-02/2016-05-05/2016-05-09/2016-05-12/2016-05-16/2016-05-19/2016-05-23/2016-05-26/2016-05-30"
date_string["201606"] = "2016-06-02/2016-06-06/2016-06-09/2016-06-13/2016-06-16/2016-06-20/2016-06-23/2016-06-27/2016-06-30"
date_string["201607"] = "2016-07-04/2016-07-07/2016-07-11/2016-07-14/2016-07-18/2016-07-21/2016-07-25/2016-07-28"
date_string["201608"] = "2016-08-01/2016-08-04/2016-08-08/2016-08-11/2016-08-15/2016-08-18/2016-08-22/2016-08-25/2016-08-29"
date_string["201609"] = "2016-09-01/2016-09-05/2016-09-08/2016-09-12/2016-09-15/2016-09-19/2016-09-22/2016-09-26/2016-09-29"
date_string["201610"] = "2016-10-03/2016-10-06/2016-10-10/2016-10-13/2016-10-17/2016-10-20/2016-10-24/2016-10-27/2016-10-31"
date_string["201611"] = "2016-11-03/2016-11-07/2016-11-10/2016-11-14/2016-11-17/2016-11-21/2016-11-24/2016-11-28"
date_string["201612"] = "2016-12-01/2016-12-05/2016-12-08/2016-12-12/2016-12-15/2016-12-19/2016-12-22/2016-12-26/2016-12-29"

date_string["201701"] = "2017-01-02/2017-01-05/2017-01-09/2017-01-12/2017-01-16/2017-01-19/2017-01-23/2017-01-26/2017-01-30"
date_string["201702"] = "2017-02-02/2017-02-06/2017-02-09/2017-02-13/2017-02-16/2017-02-20/2017-02-23/2017-02-27"
date_string["201703"] = "2017-03-02/2017-03-06/2017-03-09/2017-03-13/2017-03-16/2017-03-20/2017-03-23/2017-03-27/2017-03-30"
date_string["201704"] = "2017-04-03/2017-04-06/2017-04-10/2017-04-13/2017-04-17/2017-04-20/2017-04-24/2017-04-27"
date_string["201705"] = "2017-05-01/2017-05-04/2017-05-08/2017-05-11/2017-05-15/2017-05-18/2017-05-22/2017-05-25/2017-05-29"
date_string["201706"] = "2017-06-01/2017-06-05/2017-06-08/2017-06-12/2017-06-15/2017-06-19/2017-06-22/2017-06-26/2017-06-29"
date_string["201707"] = "2017-07-03/2017-07-06/2017-07-10/2017-07-13/2017-07-17/2017-07-20/2017-07-24/2017-07-27/2017-07-31"
date_string["201708"] = "2017-08-03/2017-08-07/2017-08-10/2017-08-14/2017-08-17/2017-08-21/2017-08-24/2017-08-28/2017-08-31"
date_string["201709"] = "2017-09-04/2017-09-07/2017-09-11/2017-09-14/2017-09-18/2017-09-21/2017-09-25/2017-09-28"
date_string["201710"] = "2017-10-02/2017-10-05/2017-10-09/2017-10-12/2017-10-16/2017-10-19/2017-10-23/2017-10-26/2017-10-30"
date_string["201711"] = "2017-11-02/2017-11-06/2017-11-09/2017-11-13/2017-11-16/2017-11-20/2017-11-23/2017-11-27/2017-11-30"
date_string["201712"] = "2017-12-04/2017-12-07/2017-12-11/2017-12-14/2017-12-18/2017-12-21/2017-12-25/2017-12-28"
########################################################################################

In [4]:
# This function generates the basic namelist dictionary for retrieving datasets
def namelist_control(target_month, forecast_type):
    namelist_dict = {# Specify the ERA-Interim data archive. Don't change.
                    "class": "s2",
                    "dataset": "s2s",
                    # Specify the date of request data
                    "date" : date_string["{}".format(target_month)],
                    "expver": "prod",
                    # in 0.75 degrees lat/lon
                    #"grid": "0.75/0.75",
                    # pressure levels (levtype:pl), all available levels (levelist)
                    #"levelist": levelist,
                    "levtype": "sfc",
                    "model": "glob",
                    "origin": "ecmf",
                    # all available parameters, for codes see http://apps.ecmwf.int/codes/grib/param-db
                    "param": "31", # sea ice fraction
                    "step": "0-24/24-48/48-72/72-96/96-120/120-144/144-168/168-192/192-216/216-240/240-264/264-288/288-312/312-336/336-360/360-384/384-408/408-432/432-456/456-480/480-504/504-528/528-552/552-576/576-600/600-624/624-648/648-672/672-696/696-720/720-744/744-768",
                    "stream": "enfo",
                    "time" : "00:00:00",
                    # reanalysis (type:an), forecast (type:fc), S2S forecast control (type:cf), S2S forecast ensemble (type:pf)
                    "type": "cf",
                    # optionally restrict area to Europe (in N/W/S/E).
                    #"area": "90/-180/30/180",
                    # Optionally get output in NetCDF format. However, this does not work with the above request due to overlapping timestamps.
                    #"format" :  "netcdf",
                    # set an output file name
                    "target": os.path.join(output_path,"S2S_sic_{}_{}.grib".format(forecast_type, target_month)),}
    
    return namelist_dict

def namelist_ensemble(target_month, forecast_type):
    namelist_dict = {# Specify the ERA-Interim data archive. Don't change.
                    "class": "s2",
                    "dataset": "s2s",
                    # Specify the date of request data
                    "date" : date_string["{}".format(target_month)],
                    "expver": "prod",
                    # in 0.75 degrees lat/lon
                    #"grid": "0.75/0.75",
                    # pressure levels (levtype:pl), all available levels (levelist)
                    #"levelist": levelist,
                    "levtype": "sfc",
                    "model": "glob",
                    "number": "1/2/3/4/5/6/7/8/9/10/11/12/13/14/15/16/17/18/19/20/21/22/23/24/25/26/27/28/29/30/31/32/33/34/35/36/37/38/39/40/41/42/43/44/45/46/47/48/49/50",
                    "origin": "ecmf",
                    # all available parameters, for codes see http://apps.ecmwf.int/codes/grib/param-db
                    "param": "31", # sea ice fraction
                    "step": "0-24/24-48/48-72/72-96/96-120/120-144/144-168/168-192/192-216/216-240/240-264/264-288/288-312/312-336/336-360/360-384/384-408/408-432/432-456/456-480/480-504/504-528/528-552/552-576/576-600/600-624/624-648/648-672/672-696/696-720/720-744/744-768",
                    "stream": "enfo",
                    "time" : "00:00:00",
                    # reanalysis (type:an), forecast (type:fc), S2S forecast control (type:cf), S2S forecast ensemble (type:pf)
                    "type": "pf",
                    # optionally restrict area to Europe (in N/W/S/E).
                    #"area": "90/-180/30/180",
                    # Optionally get output in NetCDF format. However, this does not work with the above request due to overlapping timestamps.
                    #"format" :  "netcdf",
                    # set an output file name
                    "target": os.path.join(output_path,"S2S_sic_ens_{}_{}.grib".format(forecast_type, target_month)),}
    
    return namelist_dict

In [5]:
if __name__=="__main__":
    if forecast_type == "ensemble":
        for i in target_time:
            # download data from ECMWF server
            server = ECMWFDataServer()
            # make a history log to monitor the downloading progress
            logging.info('Start Downloading SIC in {} from {} run.'.format(target_time, forecast_type))
            server.retrieve(namelist_ensemble(i, forecast_type))
            logging.info("Downloading successfully")
    else:
        for i in target_time:
            # download data from ECMWF server
            server = ECMWFDataServer()
            server.retrieve(namelist_control(i, forecast_type))
            logging.info("Downloading successfully")
            
    # record the progress in the history file
    logging.info('Finish Downloading in %s minutes' % ((tttt.time() - start_time)/60))

    print ("--- %s minutes ---" % ((tttt.time() - start_time)/60))

2020-07-20 12:52:11 ECMWF API python library 1.5.4
2020-07-20 12:52:11 ECMWF API at https://apps.ecmwf.int/v1
2020-07-20 12:52:12 Welcome Yang Liu
2020-07-20 12:52:12 In case of problems, please check https://confluence.ecmwf.int/display/WEBAPI/Web+API+FAQ or contact servicedesk@ecmwf.int
2020-07-20 12:52:12 Request submitted
2020-07-20 12:52:12 Request id: 5f15775c906598e6ada70671
2020-07-20 12:52:12 Request is submitted
2020-07-20 12:52:13 Request is active
Calling 'nice mars /tmp/20200720-1050/d3/tmp-_mars6ZEO_v.req'
mars - WARN -
mars - WARN - From 29 January 2019 10AM (UTC) MARS uses the interpolation
mars - WARN - provided by the MIR library. For more details, see
mars - WARN - https://confluence.ecmwf.int/display/UDOC/MARS+interpolation+with+MIR
mars - WARN -
MIR environment variables:
MIR_CACHE_PATH=/data/ec_coeff
mars - INFO   - 20200720.105213 - Welcome to MARS
mars - INFO   - 20200720.105213 - MARS Client bundle version: 6.28.4.0
mars - INFO   - 20200720.105213 - MARS Client